# (Preparation)

These cells verify that some dependencies are available.

# Euclidean Vectors in 2 Dimensions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
soa =np.array( [ [0,0,3,2], [0,0,1,1],[0,0,9,9]]) 
X,Y,U,V = zip(*soa)
plt.figure()
ax = plt.gca()
ax.quiver(X,Y,U,V,angles='xy',scale_units='xy',scale=1)
ax.set_xlim([-1,10])
ax.set_ylim([-1,10])
plt.draw()
plt.show()

# A Pythonic Card Deck

Let's code a package to represent decks of playing cards. First, we'll create a simple class to represent an individual card. It's so simple we just need two data attributes and no methods. Python has a factory to make such simple classes: ``collections.namedtuple``.

In [7]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

Note that the ``namedtuple`` factory takes two arguments: the name of the class to create and a sequence of attribute names.

The ``Card`` class can then be instantiated as usual.

In [8]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

The string representation of a ``Card`` instance is so explicit that you can clone a card by applying ``eval()`` to its `repr()`:

In [9]:
my_card = eval(repr(beer_card))
my_card == beer_card

True

We are now ready to code the class to represent a deck of cards. I'll call it `` FrenchDeck`` since that is the formal name of the set of 52 cards with 4 suits used not only in France but in most of the Western world.

In [10]:
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

The code is short but it does a lot.

The trickiest part is the use of a list comprehension in the initializer to build a list of cards by computing the cartesian product of the lists suits and ranks. The logic of that list comprehension is explained in chapter 2 of Fluent Python, but right now we just want to focus on the external behavior of the class, so please believe that ``self._cards`` holds a list of 52 ``Card`` instances, as you'll see right away.

In [11]:
deck = FrenchDeck()

len(deck)

52

The ``len`` built-in function knows how to handle a ``FrenchDeck`` because we implemented the ``__len__`` special method.

The `__getitem__` special method supports the use of ``[]`` and provides a lot of functionality.

We can get any card by index, as usual. For example, first and last.

In [12]:
deck[0], deck[-1]

(Card(rank='2', suit='spades'), Card(rank='A', suit='hearts'))

We can also use slice notation to retrieve a subset of the cards:

In [13]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

Here's how to use the the third parameter of a slice to get just the Aces in the deck by starting at card index 12 and skipping 13 cards from that point onwards.

In [14]:
deck[12::13]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

The `in` operator also works with our `FrenchDeck` instances. This behavior can be optimized by implementing a `__contains__` method, but if you provide a `__getitem__` method, Python is smart enough to scan the collection from item 0 to the end. 

In [15]:
Card('Q', 'hearts') in deck

True

In [16]:
Card('Z', 'clubs') in deck

False

Our card decks are also iterable. Again, implementing an `__iter__` method is the optimal way do achieve this, but as a fallback, Python knows how to iterate over any collection that offers `__getitem__` and has integer indexes starting at 0:

In [17]:
for card in deck:
  print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
Card(rank='7', suit='spades')
Card(rank='8', suit='spades')
Card(rank='9', suit='spades')
Card(rank='10', suit='spades')
Card(rank='J', suit='spades')
Card(rank='Q', suit='spades')
Card(rank='K', suit='spades')
Card(rank='A', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='3', suit='diamonds')
Card(rank='4', suit='diamonds')
Card(rank='5', suit='diamonds')
Card(rank='6', suit='diamonds')
Card(rank='7', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='A', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='8', sui

By supporting iteration, we can leverage many functions in the standard library that work with iterables, like `enumerate()`, `reversed()` as well as the constructor for `list` and several other collection types.

In [18]:
list(enumerate(reversed(deck), 1))

[(1, Card(rank='A', suit='hearts')),
 (2, Card(rank='K', suit='hearts')),
 (3, Card(rank='Q', suit='hearts')),
 (4, Card(rank='J', suit='hearts')),
 (5, Card(rank='10', suit='hearts')),
 (6, Card(rank='9', suit='hearts')),
 (7, Card(rank='8', suit='hearts')),
 (8, Card(rank='7', suit='hearts')),
 (9, Card(rank='6', suit='hearts')),
 (10, Card(rank='5', suit='hearts')),
 (11, Card(rank='4', suit='hearts')),
 (12, Card(rank='3', suit='hearts')),
 (13, Card(rank='2', suit='hearts')),
 (14, Card(rank='A', suit='clubs')),
 (15, Card(rank='K', suit='clubs')),
 (16, Card(rank='Q', suit='clubs')),
 (17, Card(rank='J', suit='clubs')),
 (18, Card(rank='10', suit='clubs')),
 (19, Card(rank='9', suit='clubs')),
 (20, Card(rank='8', suit='clubs')),
 (21, Card(rank='7', suit='clubs')),
 (22, Card(rank='6', suit='clubs')),
 (23, Card(rank='5', suit='clubs')),
 (24, Card(rank='4', suit='clubs')),
 (25, Card(rank='3', suit='clubs')),
 (26, Card(rank='2', suit='clubs')),
 (27, Card(rank='A', suit='diamo

Another powerful function that works with iterables is `sorted`. It builds a sorted list from any iterable that generates a series of comparable values (as long it fits in memory, of course).

We can define custom sorting criteria by implementing a function to produce a key from each item in the series, and passing it as the ``key==`` argument to `sorted`. Here is a function that implements the "spades high" ordering, where cards are sorted by rank and, within each rank, spades is the highest suit, followed by hearts, diamonds and clubs:

In [19]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

As written, `spades_high` produces the highest key value (51) for the Ace of spades and the lowest is the 2 of clubs:

In [20]:
spades_high(Card('2', 'clubs')), spades_high(Card('A', 'spades'))

(0, 51)

In [21]:
sorted(deck, key=spades_high)

[Card(rank='2', suit='clubs'),
 Card(rank='2', suit='diamonds'),
 Card(rank='2', suit='hearts'),
 Card(rank='2', suit='spades'),
 Card(rank='3', suit='clubs'),
 Card(rank='3', suit='diamonds'),
 Card(rank='3', suit='hearts'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='clubs'),
 Card(rank='4', suit='diamonds'),
 Card(rank='4', suit='hearts'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='clubs'),
 Card(rank='5', suit='diamonds'),
 Card(rank='5', suit='hearts'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='clubs'),
 Card(rank='6', suit='diamonds'),
 Card(rank='6', suit='hearts'),
 Card(rank='6', suit='spades'),
 Card(rank='7', suit='clubs'),
 Card(rank='7', suit='diamonds'),
 Card(rank='7', suit='hearts'),
 Card(rank='7', suit='spades'),
 Card(rank='8', suit='clubs'),
 Card(rank='8', suit='diamonds'),
 Card(rank='8', suit='hearts'),
 Card(rank='8', suit='spades'),
 Card(rank='9', suit='clubs'),
 Card(rank='9', suit='diamonds'),
 Card(rank='9', suit='hearts'),
